In [1]:
from train_semi_auto import BracketFlowModule
from parenthesis import BracketDataset
from sampling import semiauto_euler_sampling


In [2]:
checkpoint_path = "checkpoints/bracket-flow/last-v48.ckpt"
model = BracketFlowModule.load_from_checkpoint(checkpoint_path)

In [10]:
samples, trace = semiauto_euler_sampling(model, model.interpolant, 2000, 0, 3, 1000, 64)

/n/netscratch/albergo_lab/Lab/brianlck/interpretable-flow/model/transformer.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/n/netscratch/albergo_lab/Lab/brianlck/interpretable-flow/model/transformer.py:167: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


In [42]:
id = 1
dt = 1. / 2000

last = ""

e = []

for i in range(len(trace)):
    t = dt * i
    s = BracketDataset.parse_tensor(trace[i][0])
    

    for j, (a, b) in enumerate(zip(last, s)):
        if a != b:
            e.append(dict(
                t=t,
                a="c",
                tk=b,
                i=j
            ))
    
    
    if len(s) > len(last):
        for j in range(len(last), len(s)):
            e.append(dict(
                t=t,
                a="i",
                tk=s[j],
                i=j
            ))

    last = s

In [37]:
import json

json.dumps(e)

'[{"t": 0.0545, "a": "i", "tk": "m", "i": 0}, {"t": 0.11900000000000001, "a": "i", "tk": "m", "i": 1}, {"t": 0.253, "a": "i", "tk": "m", "i": 2}, {"t": 0.2745, "a": "c", "tk": ")"}, {"t": 0.3065, "a": "c", "tk": "("}, {"t": 0.3255, "a": "i", "tk": "m", "i": 3}, {"t": 0.3355, "a": "i", "tk": "m", "i": 4}, {"t": 0.3375, "a": "i", "tk": "m", "i": 5}, {"t": 0.3645, "a": "i", "tk": "m", "i": 6}, {"t": 0.424, "a": "i", "tk": "m", "i": 7}, {"t": 0.4825, "a": "i", "tk": "m", "i": 8}, {"t": 0.4855, "a": "i", "tk": "m", "i": 9}, {"t": 0.5005000000000001, "a": "c", "tk": ")"}, {"t": 0.5455, "a": "i", "tk": "m", "i": 10}, {"t": 0.596, "a": "c", "tk": "("}, {"t": 0.6065, "a": "i", "tk": "m", "i": 11}, {"t": 0.6135, "a": "c", "tk": ")"}, {"t": 0.6385000000000001, "a": "i", "tk": "m", "i": 12}, {"t": 0.651, "a": "c", "tk": ")"}, {"t": 0.683, "a": "i", "tk": "m", "i": 13}, {"t": 0.6955, "a": "c", "tk": ")"}, {"t": 0.709, "a": "i", "tk": "m", "i": 14}, {"t": 0.713, "a": "c", "tk": "("}, {"t": 0.7315, "

In [43]:
from IPython.display import display, HTML
display(HTML(f'''
<style>
.token{{display:inline-block;padding:5px 10px;margin:2px;border:1px solid #aaa;border-radius:4px;
font-family:monospace;transition:background .5s}}
.hi-i{{background:lightgreen}}
.hi-c{{background:yellow}}
#s{{min-height:30px;margin-bottom:20px}}
</style>
<div id="s"></div>
<button id="r">Replay</button>
<script>
// Use events provided from Python
var ev = {json.dumps(e)}, s = [];
var render = function(hIdx, act){{
  var c = document.getElementById("s"); c.innerHTML="";
  s.forEach(function(t,k){{ 
    var sp = document.createElement("span");
    sp.className = "token" + (k===hIdx ? " hi-"+act : "");
    sp.textContent = t; c.appendChild(sp);
  }}); 
}},
run = function(){{
  s = []; render();
  ev.forEach(function(e){{
    setTimeout(function(){{
      if(e.a=="i"){{
        var p = e.i;
        s.splice(p, 0, e.tk);
        render(p, "i");
      }} else {{
        var p = (e.i===undefined || e.i<0 || e.i>=s.length) ? Math.floor(s.length/2) : e.i;
        s[p] = e.tk; render(p, "c");
      }}
      setTimeout(render,600);
    }}, e.t*5000);
  }});
}};
document.getElementById("r").onclick = run;
run();
</script>
'''))